In [11]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

In [12]:
# Load
X = np.load('/home/filsave/malviso/X.npy', allow_pickle=True)
y = np.load('/home/filsave/malviso/y.npy', allow_pickle=True)

In [13]:
def get_values(nfeatures, X_train, X_test, y_train, y_test):
    detector = cv2.ORB_create(nfeatures)

    total_kps_train= []
    total_des_train= []
    total_kps_test= []
    total_des_test= []

    for image in X_train:
        query_kps, query_des = detector.detectAndCompute(image, None)
        total_kps_train.append(query_kps)
        total_des_train.append(query_des)

    for image in X_test:
        query_kps, query_des = detector.detectAndCompute(image, None)
        total_kps_test.append(query_kps)
        total_des_test.append(query_des)


    null_indexes_train = []

    for i,e in enumerate(total_des_train):
        if e is None:
            null_indexes_train.append(i)

    total_des_train = np.delete(total_des_train, null_indexes_train).tolist()
    y_train = np.delete(y_train,null_indexes_train).tolist()



    null_indexes_test = []

    for i,e in enumerate(total_des_test):
        if e is None:
            null_indexes_test.append(i)

    total_des_test = np.delete(total_des_test, null_indexes_test).tolist() 
    y_test = np.delete(y_test,null_indexes_test).tolist()



    max0 = 0
    for des in total_des_train:
        if des is not None:
            if des.shape[0] > max0:
                max0 = des.shape[0]

    max1=0
    for des in total_des_test:
        if des is not None:
            if des.shape[0] > max1:
                max1 = des.shape[0]
        
    if max1 > max0:
        max0=max1



    i = 0
    for des in total_des_train:
        padding_to_add = max0 - des.shape[0]
        if padding_to_add != 0:
            pad_list = []
            for k in range(32):
                pad_list.append(0)
            for j in range(padding_to_add):
                total_des_train[i] = np.append(total_des_train[i], pad_list)
            total_des_train[i] = np.array(total_des_train[i]).reshape(max0,32)
        i += 1

    i = 0
    for des in total_des_test:
        padding_to_add = max0 - des.shape[0]
        if padding_to_add != 0:
            pad_list = []
            for k in range(32):
                pad_list.append(0)
            for j in range(padding_to_add):
                total_des_test[i] = np.append(total_des_test[i], pad_list)
            total_des_test[i] = np.array(total_des_test[i]).reshape(max0,32)
        i += 1


    i = 0
    for des in total_des_train:
        total_des_train[i] = np.hstack(des)
        i += 1

    i = 0
    for des in total_des_test:
        total_des_test[i] = np.hstack(des)
        i += 1

    return total_des_train, total_des_test, y_train, y_test

In [15]:
from sklearn.metrics import f1_score

for par in range(25,45):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    total_des_train, total_des_test, y_train, y_test = get_values(par, X_train, X_test, y_train, y_test)

    clf = RandomForestClassifier(n_estimators=200, random_state=42)
    clf.fit(total_des_train, y_train)

    # predict the target values for the test data
    y_pred = clf.predict(total_des_test)

    f1 = f1_score(y_test, y_pred, average='macro')
    print(par, " f1_score accuracy: ", f1)

25  f1_score accuracy:  0.733364610878899
26  f1_score accuracy:  0.7319377215707663
27  f1_score accuracy:  0.7412207913022792
28  f1_score accuracy:  0.7450296740126208
29  f1_score accuracy:  0.7290896564375042
